<a href="https://colab.research.google.com/github/vithushanms/micrograd-backpropagation/blob/main/%5BArtificial_Neural_Networks%5D_%5BExercise%5D_Backpropagation_with_PyTourch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>